In [2]:
import pandas as pd
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
X_all=np.load("drive/MyDrive/speaker/train_gray.npy")

X_all = X_all.astype('float16')
X_all = X_all/255

Y_all=pd.read_csv("drive/MyDrive/speaker/filelist.csv",index_col=0)

X_all.shape

Y_all.shape

test=np.loadtxt("drive/MyDrive/speaker/testing_list.txt",dtype="str")

test=np.char.replace(test,".wav","")

test=np.char.split(test,"/")

test_name=np.array([x[1] for x in test])
test_type=np.array([x[0] for x in test])

valid=np.loadtxt("drive/MyDrive/speaker/validation_list.txt",dtype="str")

valid=np.char.replace(valid,".wav","")

valid=np.char.split(valid,"/")

valid_name=np.array([x[1] for x in valid])
valid_type=np.array([x[0] for x in valid])

Y_all.Name=Y_all.Name.str.replace(".png","")

Y_all["Index"]=Y_all.index

valid=pd.DataFrame({"Name":valid_name,"Type":valid_type})

test=pd.DataFrame({"Name":test_name,"Type":test_type})

valid=pd.merge(Y_all,valid,on=["Name","Type"],how="right")

test=pd.merge(Y_all,test,on=["Name","Type"],how="right")

test.Index.isna().any()

valid.Index.isna().any()

x_val=X_all[valid.Index]

y_val=Y_all.iloc[valid.Index].Type.to_numpy()

x_test=X_all[test.Index]

y_test=Y_all.iloc[test.Index].Type.to_numpy()

Y_all=Y_all[~Y_all.Index.isin(valid.Index)]
Y_all=Y_all[~Y_all.Index.isin(test.Index)]

x_train=X_all[Y_all.Index]

y_train=Y_all.Type.to_numpy()

classes=np.unique(y_train)

del X_all

del Y_all

In [5]:
commands=["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]

In [6]:
classes[np.isin(classes,commands,invert=True)]="unknown"

In [7]:
y_test[np.isin(y_test,commands,invert=True)]="unknown"

In [8]:
y_val[np.isin(y_val,commands,invert=True)]="unknown"

In [9]:
y_train[np.isin(y_train,commands,invert=True)]="unknown"

In [10]:
from sklearn import preprocessing
pre = preprocessing.LabelEncoder()
pre.fit(y_train)
y_train = pre.transform(y_train)
y_test= pre.transform(y_test)
y_val= pre.transform(y_val)

In [11]:
y_train = keras.utils.to_categorical(y_train, num_classes=11)
y_val = keras.utils.to_categorical(y_val, num_classes=11)
y_test = keras.utils.to_categorical(y_test, num_classes=11)

In [12]:
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = False
num_classes = 11
subtract_pixel_mean = True


In [13]:
n=2
depth = n * 6 + 2


In [15]:
input_shape = x_train.shape[1:]
input_shape

(99, 161, 1)

In [17]:
if subtract_pixel_mean:
    train_mean = np.mean(x_train, axis=0)
    x_train -= train_mean
    x_val -= train_mean
    x_test -= train_mean

In [18]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [19]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [20]:
def resnet_v1(input_shape, depth, num_classes=11):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model



In [21]:
model = resnet_v1(input_shape=input_shape, depth=depth)


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()



Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 99, 161, 1)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 99, 161, 16)  160         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 99, 161, 16)  64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 99, 161, 16)  0           batch_normalization[0][0]        
________________________________________________________________________

In [22]:
import tensorflow as tf
checkpoint_filepath = 'drive/MyDrive/speaker/checkpoint_model_resnet'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]





In [ ]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_val, y_val),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(train, y_train, batch_size=128),
                        validation_data=(valid, y_valid),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

Not using data augmentation.
Epoch 1/200
Learning rate:  0.001
1597/1597 [==============================] - 168s 84ms/step - loss: 1.1380 - accuracy: 0.7125 - val_loss: 0.6135 - val_accuracy: 0.8433
Epoch 2/200
Learning rate:  0.001
1597/1597 [==============================] - 135s 84ms/step - loss: 0.3542 - accuracy: 0.9212 - val_loss: 0.6731 - val_accuracy: 0.8310
Epoch 3/200
Learning rate:  0.001
1597/1597 [==============================] - 134s 84ms/step - loss: 0.2704 - accuracy: 0.9468 - val_loss: 0.3415 - val_accuracy: 0.9176
Epoch 4/200
Learning rate:  0.001
1597/1597 [==============================] - 135s 84ms/step - loss: 0.2324 - accuracy: 0.9571 - val_loss: 0.5319 - val_accuracy: 0.8691
Epoch 5/200
Learning rate:  0.001
1597/1597 [==============================] - 135s 85ms/step - loss: 0.2050 - accuracy: 0.9629 - val_loss: 0.3257 - val_accuracy: 0.9278
Epoch 6/200
Learning rate:  0.001
1597/1597 [==============================] - 135s 84ms/step - loss: 0.1856 - accuracy: 

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((valid, y_valid))